# Fourier transform of mount errors

Craig Lage - 27-Nov-22

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler

import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from astro_metadata_translator import ObservationInfo
from lsst_efd_client import merge_packed_time_series as mpts
from scipy.interpolate import UnivariateSpline, LSQUnivariateSpline

from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

In [ ]:
client = EfdClient('idf_efd')
old_butler = Butler('/repo/main', collections="LATISS/raw/all")
new_butler = Butler('/repo/embargo', collections="LATISS/raw/all")

In [ ]:
NON_TRACKING_IMAGE_TYPES = ['BIAS',
                            'FLAT',
                            ]

AUXTEL_ANGLE_TO_EDGE_OF_FIELD_ARCSEC = 280.0
MOUNT_IMAGE_WARNING_LEVEL = .25  # this determines the colouring of the cells in the table, yellow for this
MOUNT_IMAGE_BAD_LEVEL = .4


def _getEfdData(client, dataSeries, startTime, endTime):
    """A synchronous warpper for geting the data from the EFD.

    This exists so that the top level functions don't all have to be async def.
    """
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(client.select_time_series(dataSeries, ['*'], startTime.utc, endTime.utc))

In [ ]:
def findSpline(xf, yf, numKnots=18):
    s1 = 1; s2 = 1.0E7
    spline = UnivariateSpline(xf, yf, s=s1)
    knots = spline.get_knots()
    if len(knots) < numKnots:
        return s1, knots
    count = 0
    while count < 50:
        count += 1
        s = np.sqrt(s1 * s2)
        spline = UnivariateSpline(xf, yf, s=s)
        knots = spline.get_knots()
        if len(knots) > numKnots:
            s1 = s
        else:
            s2 = s
        if abs(len(knots) - numKnots) < 2:
            return s, knots
    return s,knots


In [ ]:
def calculateFFTPeaks(dataId, butler, client, limit=0.25):

    expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
    dayString = dayObsIntToString(expRecord.day_obs)
    seqNumString = str(expRecord.seq_num)
    dataIdString = f"{dayString} - seqNum {seqNumString}"

    imgType = expRecord.observation_type.upper()
    if imgType in NON_TRACKING_IMAGE_TYPES:
        return False

    exptime = expRecord.exposure_time
    if exptime < 1.99:
        return False

    tStart = expRecord.timespan.begin.tai.to_value("isot")
    tEnd = expRecord.timespan.end.tai.to_value("isot")
    elevation = 90.0 - expRecord.zenith_angle

    # TODO: DM-33859 remove this once it can be got from the expRecord
    md = butler.get('raw.metadata', dataId, detector=0)
    obsInfo = ObservationInfo(md)
    azimuth = obsInfo.altaz_begin.az.value
    # Time base in the EFD is still a big mess.  Although these times are in
    # UTC, it is necessary to tell the code they are in TAI. Then it is
    # necessary to tell the merge_packed_time_series to use UTC.
    # After doing all of this, there is still a 2 second offset,
    # which is discussed in JIRA ticket DM-29243, but not understood.

    t_start = Time(tStart, scale='tai')
    t_end = Time(tEnd, scale='tai')

    mount_position = _getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", t_start, t_end)
    nasmyth_position = _getEfdData(client, "lsst.sal.ATMCS.mount_Nasmyth_Encoders", t_start, t_end)
    torques = _getEfdData(client, "lsst.sal.ATMCS.measuredTorque", t_start, t_end)

    az = mpts(mount_position, 'azimuthCalculatedAngle', stride=1)
    el = mpts(mount_position, 'elevationCalculatedAngle', stride=1)
    rot = mpts(nasmyth_position, 'nasmyth2CalculatedAngle', stride=1)
    az_torque_1 = mpts(torques, 'azimuthMotor1Torque', stride=1)
    az_torque_2 = mpts(torques, 'azimuthMotor2Torque', stride=1)
    el_torque = mpts(torques, 'elevationMotorTorque', stride=1)
    rot_torque = mpts(torques, 'nasmyth2MotorTorque', stride=1)

    # Calculate the tracking errors
    az_vals = np.array(az.values[:, 0])
    el_vals = np.array(el.values[:, 0])
    rot_vals = np.array(rot.values[:, 0])
    times = np.array(az.values[:, 1])
    # The fits are much better if the time variable
    # is centered in the interval
    fit_times = times - times[int(len(az.values[:, 1]) / 2)]

    # Fit with a polynomial
    az_fit = np.polyfit(fit_times, az_vals, 4)
    el_fit = np.polyfit(fit_times, el_vals, 4)
    rot_fit = np.polyfit(fit_times, rot_vals, 2)
    az_model = np.polyval(az_fit, fit_times)
    el_model = np.polyval(el_fit, fit_times)
    rot_model = np.polyval(rot_fit, fit_times)

    # Errors in arcseconds
    az_error = (az_vals - az_model) * 3600
    el_error = (el_vals - el_model) * 3600
    rot_error = (rot_vals - rot_model) * 3600

    # Calculate RMS
    az_rms = np.sqrt(np.mean(az_error * az_error))
    el_rms = np.sqrt(np.mean(el_error * el_error))
    rot_rms = np.sqrt(np.mean(rot_error * rot_error))

    # Calculate Image impact RMS
    image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
    image_el_rms = el_rms
    image_rot_rms = rot_rms * AUXTEL_ANGLE_TO_EDGE_OF_FIELD_ARCSEC * np.pi / 180.0 / 3600.0
    tot_rms = np.sqrt(image_az_rms**2 + image_el_rms**2 + image_rot_rms**2)

    if tot_rms < limit:
        return False
    else:
        # Calculate the FFT peaks
        fft_peaks = []
        for i, error in enumerate([az_error, el_error]):
            # Number of samples in normalized_tone
            N = len(error)
            SAMPLE_RATE = 100 # Samples/sec
            
            yf = fft(error)
            yf = yf[0:int(len(az_error)/2)]
            xf = fftfreq(N, 1 / SAMPLE_RATE)
            xf = xf[0:int(len(error)/2)]
            yf = np.abs(fft(error))
            yf = yf[0:int(len(error)/2)]
            max = np.max(yf)
            peak_indices, peak_dict = find_peaks(yf, height=max/100) 
            peak_heights = peak_dict['peak_heights']
            
            for j in range(1,4):
                peak_index = peak_indices[np.argpartition(peak_heights,-j)[-j]]
                peak_freq = xf[peak_index]
                height_index = np.where(peak_indices == peak_index)[0][0]
                peak_height = peak_heights[height_index]
                fft_peaks.append([peak_freq, peak_height])
    return [tot_rms, fft_peaks]

    


In [ ]:
def calculateFFTKnots(dataId, butler, client, limit=0.25):
    expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
    dayString = dayObsIntToString(expRecord.day_obs)
    seqNumString = str(expRecord.seq_num)
    dataIdString = f"{dayString} - seqNum {seqNumString}"

    imgType = expRecord.observation_type.upper()
    if imgType in NON_TRACKING_IMAGE_TYPES:
        return False

    exptime = expRecord.exposure_time
    if exptime < 4.99:
        return False

    tStart = expRecord.timespan.begin.tai.to_value("isot")
    tEnd = expRecord.timespan.end.tai.to_value("isot")
    elevation = 90.0 - expRecord.zenith_angle

    # TODO: DM-33859 remove this once it can be got from the expRecord
    md = butler.get('raw.metadata', dataId, detector=0)
    obsInfo = ObservationInfo(md)
    azimuth = obsInfo.altaz_begin.az.value

    t_start = Time(tStart, scale='tai')
    t_end = Time(tEnd, scale='tai')

    mount_position = _getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", t_start, t_end)

    az = mpts(mount_position, 'azimuthCalculatedAngle', stride=1)
    el = mpts(mount_position, 'elevationCalculatedAngle', stride=1)

    # Calculate the tracking errors
    az_vals = np.array(az.values[:, 0])
    el_vals = np.array(el.values[:, 0])
    times = np.array(az.values[:, 1])
    # The fits are much better if the time variable
    # is centered in the interval
    fit_times = times - times[int(len(az.values[:, 1]) / 2)]

    # Fit with a polynomial
    az_fit = np.polyfit(fit_times, az_vals, 4)
    el_fit = np.polyfit(fit_times, el_vals, 4)
    az_model = np.polyval(az_fit, fit_times)
    el_model = np.polyval(el_fit, fit_times)

    # Errors in arcseconds
    az_error = (az_vals - az_model) * 3600
    el_error = (el_vals - el_model) * 3600

    # Calculate RMS
    az_rms = np.sqrt(np.mean(az_error * az_error))
    el_rms = np.sqrt(np.mean(el_error * el_error))
    rot_rms = np.sqrt(np.mean(rot_error * rot_error))

    # Calculate Image impact RMS
    image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
    image_el_rms = el_rms
    image_rot_rms = rot_rms * AUXTEL_ANGLE_TO_EDGE_OF_FIELD_ARCSEC * np.pi / 180.0 / 3600.0
    tot_rms = np.sqrt(image_az_rms**2 + image_el_rms**2 + image_rot_rms**2)

    if tot_rms < limit:
        return [tot_rms, None]
    else:
        # Calculate the FFT knots
        numKnots = 18
        knotList = []
        for i, error in enumerate([az_error, el_error]):
            # Number of samples in normalized_tone
            N = len(error)
            SAMPLE_RATE = 100 # Samples/sec
            yf = fft(error)
            xf = fftfreq(N, 1 / SAMPLE_RATE)
            yf = np.abs(fft(error))
            # Truncate the FFT above 5 Hz
            count = 0
            for n in range(len(xf)):
                if xf[n] < 5.0:
                    count += 1
                else:
                    break
            xf = xf[0:count]
            yf = yf[0:count]

            s, knots = findSpline(xf, yf, numKnots=numKnots)
            spline = UnivariateSpline(xf, yf, s=s)
            knots = spline.get_knots()
            values = spline(knots)
            if len(knots) < numKnots:
                numToAdd = int(numKnots - len(knots))
                lenKnots = knots[-1] - knots[0]
                for n in range(numToAdd):
                    newKnot = knots[-1] + 0.01 * (n+1)
                    newValue = values[-1]
                    knots = np.append(knots, newKnot)
                    values = np.append(values, newValue)
            if len(knots) > numKnots:
                knots = np.delete(knots, [-2])
                values = np.delete(values, [-2])
            for n in range(numKnots):
                knotList.append(knots[n])
                knotList.append(values[n])

        return [tot_rms, knotList]


In [ ]:
def calculateErrorPeaks(dataId, butler, client, limit=0.25):

    expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
    dayString = dayObsIntToString(expRecord.day_obs)
    seqNumString = str(expRecord.seq_num)
    dataIdString = f"{dayString} - seqNum {seqNumString}"

    imgType = expRecord.observation_type.upper()
    if imgType in NON_TRACKING_IMAGE_TYPES:
        return False

    exptime = expRecord.exposure_time
    if exptime < 1.99:
        return False

    tStart = expRecord.timespan.begin.tai.to_value("isot")
    tEnd = expRecord.timespan.end.tai.to_value("isot")
    elevation = 90.0 - expRecord.zenith_angle

    # TODO: DM-33859 remove this once it can be got from the expRecord
    md = butler.get('raw.metadata', dataId, detector=0)
    obsInfo = ObservationInfo(md)
    azimuth = obsInfo.altaz_begin.az.value
    # Time base in the EFD is still a big mess.  Although these times are in
    # UTC, it is necessary to tell the code they are in TAI. Then it is
    # necessary to tell the merge_packed_time_series to use UTC.
    # After doing all of this, there is still a 2 second offset,
    # which is discussed in JIRA ticket DM-29243, but not understood.

    t_start = Time(tStart, scale='tai')
    t_end = Time(tEnd, scale='tai')

    mount_position = _getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", t_start, t_end)
    nasmyth_position = _getEfdData(client, "lsst.sal.ATMCS.mount_Nasmyth_Encoders", t_start, t_end)
    torques = _getEfdData(client, "lsst.sal.ATMCS.measuredTorque", t_start, t_end)

    az = mpts(mount_position, 'azimuthCalculatedAngle', stride=1)
    el = mpts(mount_position, 'elevationCalculatedAngle', stride=1)
    rot = mpts(nasmyth_position, 'nasmyth2CalculatedAngle', stride=1)
    az_torque_1 = mpts(torques, 'azimuthMotor1Torque', stride=1)
    az_torque_2 = mpts(torques, 'azimuthMotor2Torque', stride=1)
    el_torque = mpts(torques, 'elevationMotorTorque', stride=1)
    rot_torque = mpts(torques, 'nasmyth2MotorTorque', stride=1)

    # Calculate the tracking errors
    az_vals = np.array(az.values[:, 0])
    el_vals = np.array(el.values[:, 0])
    rot_vals = np.array(rot.values[:, 0])
    times = np.array(az.values[:, 1])
    # The fits are much better if the time variable
    # is centered in the interval
    fit_times = times - times[int(len(az.values[:, 1]) / 2)]

    # Fit with a polynomial
    az_fit = np.polyfit(fit_times, az_vals, 4)
    el_fit = np.polyfit(fit_times, el_vals, 4)
    rot_fit = np.polyfit(fit_times, rot_vals, 2)
    az_model = np.polyval(az_fit, fit_times)
    el_model = np.polyval(el_fit, fit_times)
    rot_model = np.polyval(rot_fit, fit_times)

    # Errors in arcseconds
    az_error = (az_vals - az_model) * 3600
    el_error = (el_vals - el_model) * 3600
    rot_error = (rot_vals - rot_model) * 3600

    # Calculate RMS
    az_rms = np.sqrt(np.mean(az_error * az_error))
    el_rms = np.sqrt(np.mean(el_error * el_error))
    rot_rms = np.sqrt(np.mean(rot_error * rot_error))

    # Calculate Image impact RMS
    image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
    image_el_rms = el_rms
    image_rot_rms = rot_rms * AUXTEL_ANGLE_TO_EDGE_OF_FIELD_ARCSEC * np.pi / 180.0 / 3600.0
    tot_rms = np.sqrt(image_az_rms**2 + image_el_rms**2 + image_rot_rms**2)

    if tot_rms < limit:
        return False
    else:
        # Calculate the error peaks
        error_peaks = []
        for i, error in enumerate([az_error, el_error]):
            max = np.max(error)
            peak_indices, peak_dict = find_peaks(error, height=max/100) 
            peak_heights = peak_dict['peak_heights']
            print(i, len(error), peak_indices, peak_heights)
            """
            for j in range(1,4):
                peak_index = peak_indices[np.argpartition(peak_heights,-j)[-j]]
                peak_freq = xf[peak_index]
                height_index = np.where(peak_indices == peak_index)[0][0]
                peak_height = peak_heights[height_index]
                fft_peaks.append([peak_freq, peak_height])
            """
    return 

    


In [ ]:
#expId = 2023110800415 # Oscillation
#expId = 2023111600552 # Wind
expId = 2023111600561 # Crazy mount?
#expId = 2023112000238 # Crazy mount?
#expId = 2023112000201 # Shutter open too soon
#expId = 2023110700594 # Timebase errors 1
#expId = 2023110700519 # Timebase errors 2
dataId = {'detector':0, 'exposure':expId}
[tot_rms, knotList] = calculateFFTKnots(dataId, new_butler, client)
print(knotList)

In [ ]:
dayObs_list = [20221110, 20221212, 20230118, 20230216, \
               20220315, 20230511, 20230817, 20231107, 20231113, 20231121, 20231128, 20231129, 20231130]

Mount_FFT_Dict = {}
for dayObs in dayObs_list:
    if dayObs < 20220915:
        butler = old_butler
    else:
        butler = new_butler

    exposureList = []

    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        if record.observation_type not in ['bias', 'flat', 'dark']:
            exposureList.append(record.id)
    exposureList = sorted(exposureList)

    for expId in exposureList:
        try:
            dataId = {'detector':0, 'exposure':expId}
            result = calculateFFTKnots(dataId, butler, client)
            #print(expId, result)
            if result:
                [tot_rms, fftKnots] = result
                resDict = {}
                resDict['Cause'] = None
                resDict['RMS'] = tot_rms
                resDict['FFT_knots'] = fftKnots
                Mount_FFT_Dict[expId] = resDict
                print(f"Finished {expId}")
        except:
            continue
    outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict_Knots.pkl', 'wb')

    pkl.dump(Mount_FFT_Dict,outfile)
    outfile.close()
    print(f"Finished {dayObs}")


In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict_Knots.pkl', 'rb')
Mount_FFT_Dict = pkl.load(infile)
infile.close()
print(list(Mount_FFT_Dict.keys())[-1])
print(len(list(Mount_FFT_Dict.keys())))

In [ ]:
for expId in list(Mount_FFT_Dict.keys()):
    if len(Mount_FFT_Dict[expId]['FFT_knots']) != 72:
        print(expId, len(Mount_FFT_Dict[expId]['FFT_knots']))


In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict.pkl', 'rb')
Mount_FFT_Dict = pkl.load(infile)
Mount_FFT_Dict_Classified = Mount_FFT_Dict.copy()
infile.close()
outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict_Classified.pkl', 'wb')
pkl.dump(Mount_FFT_Dict_Classified,outfile)
outfile.close()

In [ ]:
import webbrowser

infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict_Classified.pkl', 'rb')
Mount_FFT_Dict_Classified = pkl.load(infile)
infile.close()
causes = ['OSC', 'WIN', 'CRA', 'TIM', 'SHU']

print(causes)
for key in Mount_FFT_Dict_Classified.keys():
    year = int(key/1000000000)
    month = int((key - 1000000000 * year)/10000000)
    day = int((key - 1000000000 * year - 10000000 * month)/100000)
    seqNum = int((key - 1000000000 * year - 10000000 * month - 100000 * day))

    if Mount_FFT_Dict_Classified[key]['Cause'] is None:
        webbrowser.open(f'https://roundtable.lsst.codes/rubintv/summit/auxtel/mount/event/{year}-{month:02}-{day:02}/{seqNum}')
        break
        cause = input(f"Classification of {key}")
        if cause == 'STOP':
            break
        else:
            Mount_FFT_Dict_Classified[key]['Cause'] = cause

outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict_Classified.pkl', 'wb')
pkl.dump(Mount_FFT_Dict_Classified,outfile)
outfile.close()

In [ ]:


webbrowser.open('http://example.com')
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict_Classified.pkl', 'rb')
Mount_FFT_Dict_Classified = pkl.load(infile)
infile.close()
causes = ['OSC', 'WIN', 'CRA', 'TIM', 'SHU']

print(causes)
for key in Mount_FFT_Dict_Classified.keys():
    print(key, Mount_FFT_Dict_Classified[key]['Cause'])


In [ ]:
Mount_FFT_Dict[2022111000353]['Cause'] is None

In [ ]:
for key in Mount_FFT_Dict_Classified[2022111000235].keys():
    print(key, Mount_FFT_Dict_Classified[2022111000235][key])

In [ ]:
for key in Mount_FFT_Dict_Classified.keys():
    year = int(key/1000000000)
    month = int((key - 1000000000 * year)/10000000)
    day = int((key - 1000000000 * year - 10000000 * month)/100000)
    seqNum = int((key - 1000000000 * year - 10000000 * month - 100000 * day))
    print(year, month, day, seqNum)
    break


In [ ]:
url = f'https://roundtable.lsst.codes/rubintv/summit/auxtel/mount/event/{year}-{month:02}-{day:02}/{seqNum}'
webbrowser.open_new(url)

In [ ]:
print(f'https://roundtable.lsst.codes/rubintv/summit/auxtel/mount/event/{year}-{month:02}-{day:02}/{seqNum}')

In [ ]:
webbrowser._browsers.items()

In [ ]:
#expId = 2023110800415 # Oscillation
#expId = 2023111600552 # Wind
#expId = 2023111600561 # Crazy mount?
#expId = 2023112000238 # Crazy mount?
expId = 2023112000201 # Shutter open too soon
expId = 2023113000629 # Shutter open too soon
#expId = 2023110700594 # Timebase errors 1
#expId = 2023110700519 # Timebase errors 2
#expId = 2023122200404 # 5 second crzy mount

dataId = {'detector':0, 'exposure':expId}

butler = new_butler

expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
dayString = dayObsIntToString(expRecord.day_obs)
seqNumString = str(expRecord.seq_num)
dataIdString = f"{dayString} - seqNum {seqNumString}"


tStart = expRecord.timespan.begin.tai.to_value("isot")
tEnd = expRecord.timespan.end.tai.to_value("isot")
elevation = 90.0 - expRecord.zenith_angle

# TODO: DM-33859 remove this once it can be got from the expRecord
md = butler.get('raw.metadata', dataId, detector=0)
obsInfo = ObservationInfo(md)
azimuth = obsInfo.altaz_begin.az.value
# Time base in the EFD is still a big mess.  Although these times are in
# UTC, it is necessary to tell the code they are in TAI. Then it is
# necessary to tell the merge_packed_time_series to use UTC.
# After doing all of this, there is still a 2 second offset,
# which is discussed in JIRA ticket DM-29243, but not understood.

t_start = Time(tStart, scale='tai')
t_end = Time(tEnd, scale='tai')

mount_position = _getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", t_start, t_end)
nasmyth_position = _getEfdData(client, "lsst.sal.ATMCS.mount_Nasmyth_Encoders", t_start, t_end)
torques = _getEfdData(client, "lsst.sal.ATMCS.measuredTorque", t_start, t_end)

az = mpts(mount_position, 'azimuthCalculatedAngle', stride=1)
el = mpts(mount_position, 'elevationCalculatedAngle', stride=1)
rot = mpts(nasmyth_position, 'nasmyth2CalculatedAngle', stride=1)
az_torque_1 = mpts(torques, 'azimuthMotor1Torque', stride=1)
az_torque_2 = mpts(torques, 'azimuthMotor2Torque', stride=1)
el_torque = mpts(torques, 'elevationMotorTorque', stride=1)
rot_torque = mpts(torques, 'nasmyth2MotorTorque', stride=1)

# Calculate the tracking errors
az_vals = np.array(az.values[:, 0])
el_vals = np.array(el.values[:, 0])
rot_vals = np.array(rot.values[:, 0])
times = np.array(az.values[:, 1])
# The fits are much better if the time variable
# is centered in the interval
fit_times = times - times[int(len(az.values[:, 1]) / 2)]

# Fit with a polynomial
az_fit = np.polyfit(fit_times, az_vals, 4)
el_fit = np.polyfit(fit_times, el_vals, 4)
rot_fit = np.polyfit(fit_times, rot_vals, 2)
az_model = np.polyval(az_fit, fit_times)
el_model = np.polyval(el_fit, fit_times)
rot_model = np.polyval(rot_fit, fit_times)

# Errors in arcseconds
az_error = (az_vals - az_model) * 3600
el_error = (el_vals - el_model) * 3600
rot_error = (rot_vals - rot_model) * 3600

# Calculate RMS
az_rms = np.sqrt(np.mean(az_error * az_error))
el_rms = np.sqrt(np.mean(el_error * el_error))
rot_rms = np.sqrt(np.mean(rot_error * rot_error))

# Calculate Image impact RMS
image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
image_el_rms = el_rms
image_rot_rms = rot_rms * AUXTEL_ANGLE_TO_EDGE_OF_FIELD_ARCSEC * np.pi / 180.0 / 3600.0
tot_rms = np.sqrt(image_az_rms**2 + image_el_rms**2 + image_rot_rms**2)

# Check the timebase errors
cRIO_ts = mount_position["cRIO_timestamp"]
timestamps = cRIO_ts.values
deltaTs = []
for n in range(1, len(timestamps)):
    deltaTs.append(timestamps[n] - timestamps[n-1])
print("Mean deltaT", np.mean(deltaTs), "Max deltaT", np.max(deltaTs)) 
# Calculate the FFT peaks
fft_peaks = []
fig, axs = plt.subplots(1,2)
plt.subplots_adjust(wspace=0.5)
numKnots = 18
knotList = []
names = ["Azimuth", "Elevation"]
for i, error in enumerate([az_error, el_error]):
    # Number of samples in normalized_tone
    N = len(error)
    SAMPLE_RATE = 100 # Samples/sec
    yf = fft(error)
    xf = fftfreq(N, 1 / SAMPLE_RATE)
    yf = np.abs(fft(error))
    count = 0
    for n in range(len(xf)):
        if xf[n] < 5.0:
            count += 1
        else:
            break
    print(count)
    xf = xf[0:count]
    yf = yf[0:count]
    """            
    numKnots = 40
    knots = np.linspace(xf[1], xf[-2], numKnots)
    spline = LSQUnivariateSpline(xf, yf, knots)
    #knots = spline.get_knots()
    values = spline(knots)

    """
    numKnots = 20
    s, knots = findSpline(xf, yf, numKnots=numKnots)
    spline = UnivariateSpline(xf, yf, s=s)
    knots = spline.get_knots()
    values = spline(knots)
    #print(len(knots), len(values))
    
    if len(knots) < numKnots:
        numToAdd = int(numKnots - len(knots))
        lenKnots = knots[-1] - knots[0]
        for n in range(numToAdd):
            newKnot = knots[-1] + 0.01 * (n+1)
            newValue = values[-1]
            knots = np.append(knots, newKnot)
            values = np.append(values, newValue)
    if len(knots) > numKnots:
        knots = np.delete(knots, [-2])
        values = np.delete(values, [-2])
    axs[i].set_title(names[i])
    axs[i].plot(xf, yf, color='blue')
    axs[i].plot(xf, spline(xf), ls='--', color='red')
    axs[i].scatter(knots, values, marker='x', color='red')
    axs[i].set_xlabel("Frequency(Hz)")
    axs[i].set_ylabel("Amplitude")
    print(i, len(knots))
    for n in range(numKnots):
        knotList.append(knots[n])
        knotList.append(values[n])
print(len(knotList))
#print(knotList)
plt.savefig(f"/home/c/cslage/u/AuxTel/mount_classifier/Spline_Knots_{expId}.png")

In [ ]:
causes = ['OSC', 'WIN', 'CRA', 'TIM', 'SHU', 'GOOD', 'UNSURE']
fullCauses = ['UNSURE', 'OSC', 'WIN', 'CRA', 'TIM', 'SHU', 'GOOD'] 


In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_29Dec23.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()


In [ ]:
for cause in fullCauses:
    print(cause, Mount_Errors_Classified_Dict[20231204][cause])

In [ ]:
fullCauses = ['UNSURE', 'OSC', 'WIN', 'CRA', 'TIM', 'SHU', 'GOOD'] 

infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_29Dec23.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()

plotColors = ['gray', 'red', 'blue', 'violet', 'yellow', 'cyan', 'green']
dates = Mount_Errors_Classified_Dict.keys()
plotDates = []
plotList = []
xticks = []

skipDates = 2
# Somehow some empty lists got in
# This cleans them out
badDates = []
for date in dates:
    if len(Mount_Errors_Classified_Dict[date]['GOOD']) < 5:
        badDates.append(date)
for date in badDates:
    del Mount_Errors_Classified_Dict[date]
        
dates = Mount_Errors_Classified_Dict.keys()
data = np.zeros([len(dates), len(fullCauses)])
for i, date in enumerate(dates):
    for j, cause in enumerate(fullCauses):
        data[i,j] = len(Mount_Errors_Classified_Dict[date][cause])
for i, date in enumerate(dates):
    sum = data[i,:].sum()
    if i % skipDates == 0:
        xticks.append(str(date))
    else:
        xticks.append("")
    plotDates.append(str(date))
    for j, cause in enumerate(fullCauses):
        data[i,j] /= sum
        data[i,j] *= 100.0

for j, cause in enumerate(fullCauses):
    plotList.append(data[:,j])


fig, ax = plt.subplots(figsize=(10,8))
ax.set_title("AuxTel types of mount fails - 2023", fontsize=18)
ax.stackplot(plotDates, plotList, labels=fullCauses, colors=plotColors)
ax.legend(loc='upper left')
ax.set_ylabel("Percent")
ax.set_xticks(ticks=plotDates,labels=xticks, rotation=90)
plt.savefig('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Fails_Classified_2023.png')

In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_29Dec23.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()
dates = Mount_Errors_Classified_Dict.keys()

client = EfdClient('idf_efd') # Before 20231211  
fullCauses = ['UNSURE', 'OSC', 'WIN', 'CRA', 'TIM', 'SHU', 'GOOD'] 
from lsst.summit.utils.efdUtils import calcNextDay
windSpeeds = []
windFails = []
for date in dates:
    try:
        dayObs = int(date)
        dateTime = datetime.strptime(str(dayObs), "%Y%m%d")
        dayObsDate = f"{dateTime.year}-{dateTime.month}-{dateTime.day}"
        nextDayObs = calcNextDay(dayObs)
        dateTime = datetime.strptime(str(nextDayObs), "%Y%m%d")
        nextDayObsDate = f"{dateTime.year}-{dateTime.month}-{dateTime.day}"
        start = Time(f"{dayObsDate} 23:00:00Z", scale='utc')
        end = Time(f"{nextDayObsDate} 08:00:00Z", scale='utc')
        maxSpeed = await client.select_time_series('lsst.sal.ESS.airFlow', \
                                                ['maxSpeed'],  start, end, index=301)
        windSpeed = np.median(maxSpeed['maxSpeed'].values)
        total = 0
        for cause in fullCauses:
            total += len(Mount_Errors_Classified_Dict[date][cause])
            if cause == 'WIN':
                windFail = len(Mount_Errors_Classified_Dict[date][cause])
        windFail = windFail / total * 100.0
        windSpeeds.append(windSpeed)
        windFails.append(windFail)
    except:
        continue
print(len(windSpeeds), len(windFails))
plt.scatter(windSpeeds, windFails)
plt.title("Mount fails due to wind jitter vs median wind speed", fontsize=16)
plt.ylabel("Percent fails due to wind jitter")
plt.xlabel("Median wind speed (m/s)")
plt.savefig('/home/c/cslage/u/AuxTel/mount_classifier/Wind_Fails_vs_Wind_Speed_2023.png')